![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# The output

Your query should return a table in the following format:
| industry  | year | num\_unicorns       | average\_valuation\_billions |
| --------- | ---- | ------------------- | ---------------------------- |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.

## **Finding the top industries**

In [4]:
--Identifying the top industries
WITH top_industries AS (  									  --temporary result
  SELECT i.industry, COUNT(*) AS num_companies 				  --tbl industry as i; cnts each company
  FROM industries AS i										  --tbl from industry aliased as i
  JOIN dates AS d 											  --joins the industries tbl to dates tbl 
	ON i.company_id = d.company_id							  --with the use of company_id both tbls
  WHERE EXTRACT(year FROM d.date_joined) IN (2019, 2020, 2021)--extract func gets yr frm date_joined clm
  GROUP BY i.industry										  --groups the results by industry
  ORDER BY num_companies DESC								  --sorts the grp results in desc order
  LIMIT 3													  --limits the outpt to the top 3 indstry
)
SELECT * FROM top_industries;								  --get and display the result

,industry,num_companies
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


## **Gathering yearly rankings data**

In [5]:
--Identifying the top industries
WITH top_industries AS (
  SELECT i.industry, COUNT(*) AS num_companies
  FROM industries AS i
  JOIN dates AS d 
	ON i.company_id = d.company_id
  WHERE EXTRACT(year FROM d.date_joined) IN (2019, 2020, 2021)
  GROUP BY i.industry
  ORDER BY num_companies DESC
  LIMIT 3
),

--Gathering yearly rankings data
yearly_rankings AS (											   --2nd temp result for yearly unicorn stats
  SELECT 
    i.industry,													   --industry name from industries tbl
    EXTRACT(YEAR FROM d.date_joined) AS year,					   --extracts yr unicorn status from date_joined
    COUNT(*) AS num_unicorns,									   --cnts unicorns per industry per year
    ROUND(AVG(f.valuation) / 1000, 2) AS average_valuation_billions--avg valuation in billions, rounded
  FROM industries AS i											   --tbl from industry aliased as i
  JOIN dates AS d ON i.company_id = d.company_id				   --joins dates tbl using company_id
  JOIN funding AS f ON i.company_id = f.company_id				   --joins funding tbl using company_id
  WHERE i.industry IN (SELECT industry FROM top_industries)		   --filters only top 3 industries from 1st CTE
  GROUP BY i.industry, year										   --groups by industry and year
)

--Displaying the final result
SELECT * FROM yearly_rankings;									   --get and display the result from 2nd CTE

,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2012,1,27000000.00
1,Internet software & services,2015,4,1250000.00
2,Fintech,2018,10,8600000.00
3,Fintech,2020,15,4333333.33
4,E-commerce & direct-to-consumer,2018,4,27250000.00
5,Fintech,2011,1,46000000.00
6,E-commerce & direct-to-consumer,2020,16,4000000.00
7,Fintech,2014,1,95000000.00
8,Fintech,2015,2,5500000.00
9,E-commerce & direct-to-consumer,2017,4,1500000.00


## **Returning the final results**

In [6]:
WITH top_industries AS (
  SELECT i.industry, COUNT(*) AS num_companies
  FROM industries AS i
  JOIN dates AS d 
	ON i.company_id = d.company_id
  WHERE EXTRACT(year FROM d.date_joined) IN (2019, 2020, 2021)
  GROUP BY i.industry
  ORDER BY num_companies DESC
  LIMIT 3
),

yearly_rankings AS (
  SELECT 
    i.industry,
    EXTRACT(YEAR FROM d.date_joined) AS year,
    COUNT(*) AS num_unicorns,
    ROUND(AVG(f.valuation) / 1000, 2) AS average_valuation_billions
  FROM industries AS i
  JOIN dates AS d ON i.company_id = d.company_id
  JOIN funding AS f ON i.company_id = f.company_id
  WHERE i.industry IN (SELECT industry FROM top_industries)
  GROUP BY i.industry, year
)

SELECT 
  industry,
  year,
  num_unicorns,
  average_valuation_billions
FROM yearly_rankings
WHERE year IN (2019, 2020, 2021)										--filter by target years
  AND industry IN (SELECT industry FROM top_industries)					--filter for top industries
ORDER BY year DESC, num_unicorns DESC;									--sort by year and unicorn count

,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2753623.19
1,Internet software & services,2021,119,2151260.50
2,E-commerce & direct-to-consumer,2021,47,2468085.11
3,Internet software & services,2020,20,4350000.00
4,E-commerce & direct-to-consumer,2020,16,4000000.00
5,Fintech,2020,15,4333333.33
6,Fintech,2019,20,6800000.00
7,Internet software & services,2019,13,4230769.23
8,E-commerce & direct-to-consumer,2019,12,2583333.33
